In [1]:
from configs.task_2 import get_config
from model import ConvVAE, reparameterize
from utils.vae import save_image

import jax
import jax.numpy as jnp
from flax.training.checkpoints import restore_checkpoint

rng = jax.random.PRNGKey(42)

In [2]:
from task2_dataset import load_image, get_image_path

woman = load_image(get_image_path("task2_dataset/lfw/Aishwarya_Rai"))[0]
old_man = load_image(get_image_path("task2_dataset/lfw/Bernard_Ebbers"))[0]
rd_jr = load_image(get_image_path("task2_dataset/lfw/Robert_Downey_Jr"))[0]
leo_dc = load_image(get_image_path("task2_dataset/lfw/Leonardo_DiCaprio"))[0]
zrj = load_image(["task2_dataset/lfw/Zhu_Rongji/Zhu_Rongji_0009.jpg"])[0]
james = load_image(["task2_dataset/lfw/LeBron_James/LeBron_James_0001.jpg"])[0]
joey = load_image(get_image_path("task2_dataset/lfw/Matt_LeBlanc"))[0]
chandler = load_image(["task2_dataset/lfw/Matthew_Perry/Matthew_Perry_0007.jpg"])[0]
rachel = load_image(["task2_dataset/lfw/Jennifer_Aniston/Jennifer_Aniston_0016.jpg"])[0]

imgs = jnp.einsum("bchw->bhwc", jnp.stack([old_man, rd_jr, leo_dc, zrj, james, joey, chandler, rachel], axis=0))

In [3]:
config = get_config()
variables = restore_checkpoint("ckpts", None, prefix="task2_")
model = ConvVAE(config)

In [4]:
rng, _rng = jax.random.split(rng) 
mean, logvar = model.apply(variables, imgs, method="encode")
zs = reparameterize(_rng, mean, logvar)

In [5]:
alphas = [0.0, 0.2, 0.3, 0.5, 0.6, 0.7, 0.9, 1.0]
inps = []

for z0, z1 in zip(zs[:-1], zs[1:]):
  for alpha in alphas:
    z = alpha * z0 + (1 - alpha) * z1
    inps.append(z)

inps = jnp.array(inps)

In [6]:
imgs = model.apply(variables, inps, method="decode")
save_image(imgs, "outputs/vae_recon.pdf", nrow=8)